# JLE Supplier Quality, Retention, and Learning-Curve Analysis
**Author:** Ivanna Nguyen  

**Goal:** Recreate and extend the Excel-based case analysis in Python. We will:
- clean the three worksheets in the provided Excel file,
- build models (trend/counterfactual, multiple regression, and learning curve),
- visualize results, and
- export clean tables and charts for a portfolio/GitHub repo.

> This notebook mirrors a "clean → analyze → visualize" pattern similar to common portfolio examples.


In [ ]:
# --- setup: imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pathlib import Path

# plotting rule of thumb for portfolios: one chart per cell, no specific colors
plt.rcParams['figure.figsize'] = (10,6)


In [ ]:
# --- locate Excel file (choose the first one that exists)
excel_candidates = [
    Path('/mnt/data/Chapter 8 Case Data (1).xlsx'),
    Path('/mnt/data/Chapter 8 Case Data (2).xlsx'),
    Path('data/chapter8_case_data.xlsx'),  # fallback if running from a repo
]

excel_path = None
for p in excel_candidates:
    if p.exists():
        excel_path = p
        break

assert excel_path is not None, "Excel file not found. Update excel_candidates to point to your file."
excel_path

In [ ]:
# --- inspect sheets
xls = pd.ExcelFile(excel_path)
xls.sheet_names

In [ ]:
# quick peek at the first few rows from each sheet
sheets = {name: pd.read_excel(excel_path, sheet_name=name) for name in xls.sheet_names}
for name, df in sheets.items():
    print(f"=== {name} ===")
    print(df.head(), "\n")

## 1) Supplier Defects — Clean, Model, Visualize
We tidy the `Defects after Delivery` sheet, fit a pre‑initiative trend, and compare the **actual** path to a **counterfactual** path that assumes the initiative never happened (initiative in **August 2015**).

In [ ]:
def clean_defects(df):
    # first visible row contains 'Month, 2014, 2015, 2016, 2017, 2018'
    tidy = df.iloc[1:].copy()
    cols = ['Month'] + [str(int(x)) for x in df.iloc[0,1:].values if pd.notnull(x)]
    tidy.columns = ['Month'] + cols[1:]
    # reshape to long format
    long_df = tidy.melt(id_vars='Month', var_name='Year', value_name='DefectsPerMillion')
    long_df = long_df.dropna(subset=['Month','DefectsPerMillion'])
    # month order and time index
    month_order = {m:i for i,m in enumerate(
        ['January','February','March','April','May','June','July','August','September','October','November','December'], start=1)}
    long_df['MonthNum'] = long_df['Month'].map(month_order)
    long_df['Year'] = long_df['Year'].astype(int)
    long_df = long_df.sort_values(['Year','MonthNum']).reset_index(drop=True)
    start_year = long_df['Year'].min()
    long_df['t'] = (long_df['Year'] - start_year) * 12 + (long_df['MonthNum'] - 1) + 1
    return long_df

def analyze_defects(long_df, initiative_year=2015, initiative_month=8):
    # compute time index for initiative month
    t0 = long_df.loc[(long_df['Year']==initiative_year) & (long_df['MonthNum']==initiative_month), 't'].iloc[0]
    pre = long_df[long_df['t'] < t0]
    post = long_df[long_df['t'] >= t0]
    # fit linear trend on pre period
    X_pre = sm.add_constant(pre['t'])
    y_pre = pre['DefectsPerMillion']
    model = sm.OLS(y_pre, X_pre).fit()
    # predict counterfactual for post
    X_post = sm.add_constant(post['t'])
    post = post.copy()
    post['Counterfactual'] = model.predict(X_post)
    post['Improvement_vs_CF'] = post['Counterfactual'] - post['DefectsPerMillion']
    post['Improvement_pct'] = post['Improvement_vs_CF'] / post['Counterfactual']
    return model, t0, pre, post

defects_raw = sheets['Defects after Delivery']
defects_long = clean_defects(defects_raw)
def_model, t0, defects_pre, defects_post = analyze_defects(defects_long)

avg_improve = defects_post['Improvement_pct'].mean()
avg_improve

In [ ]:
# plot: actual path vs. counterfactual after initiative
plt.plot(defects_long['t'], defects_long['DefectsPerMillion'])
plt.plot(defects_post['t'], defects_post['Counterfactual'])
plt.axvline(t0, linestyle='--')
plt.title('supplier defects per million — actual vs pre‑trend counterfactual')
plt.xlabel('months since jan 2014')
plt.ylabel('defects per million')
plt.show()

## 2) Employee Retention — Multiple Regression
Target: `YearsPLE` (years with company)  

Predictors: `YrsEducation`, `College GPA`, `Age`

In [ ]:
ret_raw = sheets['Employee Retention'].copy()
# header row is at index 1
ret_cols = ret_raw.iloc[1].tolist()
ret_df = ret_raw.iloc[2:].copy()
ret_df.columns = ret_cols
ret_df = ret_df[['YearsPLE','YrsEducation','College GPA','Age']].dropna()
ret_df = ret_df.astype({'YearsPLE':'float','YrsEducation':'float','College GPA':'float','Age':'float'})

X = sm.add_constant(ret_df[['YrsEducation','College GPA','Age']])
y = ret_df['YearsPLE']
ret_model = sm.OLS(y, X).fit()
ret_model.rsquared

In [ ]:
# quick coefficient table
ret_model.params

In [ ]:
# viz: GPA vs. tenure (simple OLS line for display)
plt.scatter(ret_df['College GPA'], ret_df['YearsPLE'])
X_gpa = sm.add_constant(ret_df['College GPA'])
yhat = sm.OLS(y, X_gpa).fit().predict(X_gpa)
plt.plot(ret_df['College GPA'], yhat)
plt.title('employee retention vs. college gpa')
plt.xlabel('college gpa')
plt.ylabel('years with company')
plt.show()

## 3) Engine Production — Learning Curve
Model: \( \log(T) = a + b \log(N) \). Learning rate when output doubles = \(2^b\).

In [ ]:
eng_raw = sheets['Engines'].copy()
eng_df = eng_raw.iloc[2:].copy()
eng_df.columns = ['Sample','ProductionTimeMin']
eng_df = eng_df.dropna()
eng_df['Sample'] = eng_df['Sample'].astype(int)
eng_df['ProductionTimeMin'] = pd.to_numeric(eng_df['ProductionTimeMin'])

eng_df['logN'] = np.log(eng_df['Sample'])
eng_df['logT'] = np.log(eng_df['ProductionTimeMin'])
X = sm.add_constant(eng_df['logN'])
y = eng_df['logT']
lc_model = sm.OLS(y, X).fit()

b = lc_model.params['logN']
learning_rate = float(2 ** b)
learning_rate

In [ ]:
# plot: learning curve in original units
plt.scatter(eng_df['Sample'], eng_df['ProductionTimeMin'])
yhat = np.exp(lc_model.predict(X))
plt.plot(eng_df['Sample'], yhat)
plt.title('engine production learning curve')
plt.xlabel('cumulative unit number')
plt.ylabel('production time (minutes)')
plt.show()

## 4) Export Clean Tables and Visuals
These files can be committed to your repository for reviewers to explore alongside the notebook.


In [ ]:
out_dir = Path('jle_outputs')
out_dir.mkdir(exist_ok=True)

defects_post.to_csv(out_dir / 'defects_actual_vs_counterfactual.csv', index=False)
ret_df.to_csv(out_dir / 'employee_retention_clean.csv', index=False)
eng_df.to_csv(out_dir / 'engines_clean.csv', index=False)

# save the three figures again
# 1) defects
plt.figure()
plt.plot(defects_long['t'], defects_long['DefectsPerMillion'])
plt.plot(defects_post['t'], defects_post['Counterfactual'])
plt.axvline(t0, linestyle='--')
plt.title('supplier defects per million — actual vs pre‑trend counterfactual')
plt.xlabel('months since jan 2014')
plt.ylabel('defects per million')
plt.tight_layout()
plt.savefig(out_dir / 'defects_actual_vs_counterfactual.png', dpi=220)
plt.close()

# 2) retention
plt.figure()
plt.scatter(ret_df['College GPA'], ret_df['YearsPLE'])
X_gpa = sm.add_constant(ret_df['College GPA'])
yhat = sm.OLS(ret_df['YearsPLE'], X_gpa).fit().predict(X_gpa)
plt.plot(ret_df['College GPA'], yhat)
plt.title('employee retention vs. college gpa')
plt.xlabel('college gpa')
plt.ylabel('years with company')
plt.tight_layout()
plt.savefig(out_dir / 'retention_vs_gpa.png', dpi=220)
plt.close()

# 3) learning curve
plt.figure()
plt.scatter(eng_df['Sample'], eng_df['ProductionTimeMin'])
yhat = np.exp(lc_model.predict(X))
plt.plot(eng_df['Sample'], yhat)
plt.title('engine production learning curve')
plt.xlabel('cumulative unit number')
plt.ylabel('production time (minutes)')
plt.tight_layout()
plt.savefig(out_dir / 'learning_curve.png', dpi=220)
plt.close()

summary = f"""average improvement vs. counterfactual after initiative: {defects_post['Improvement_pct'].mean():.1%}
retention model R^2: {ret_model.rsquared:.3f}
estimated learning rate (time drop when doubling output): {learning_rate:.3f}
"""
with open(out_dir / 'readme_numbers.txt', 'w') as f:
    f.write(summary)

sorted([p.name for p in out_dir.iterdir()])